In [12]:
from performance import Encoder
from analysis import plot_grb_solution, compare_outputs, get_grb_inputs
import flags_constants as fc

fc.manhattan_use_absolute_value = False
fc.manhattan_use_lp_constraints = True
fc.bounds_relu_early_stopping = True

import numpy as np
% matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import pickle
from timeit import default_timer as timer
import sys

In [13]:
def iterative_solving(reference, test, input_los, input_his, mode, center, metric, radius_hi, 
                      optim_limit=120, time_limit=600, start_file='best_solution.mst', valEps=0.001):
    t_start = timer()
    now = t_start
    
    bestVal = np.inf
    mipBound = False
    solutionFound = False
    
    enc = Encoder()
    enc.encode_equiv(reference, test, input_los, input_his, mode)
    
    
    enc.add_input_radius(center, radius_hi, metric=metric, radius_mode='variable')
    
    iteration = 0
    while now - t_start < time_limit:
        t0 = timer()
        enc.optimize_constraints(metric=metric, use_lp_relaxation=True)
        t1 = timer()    
        print("Bounds calculation: {}".format(t1 - t0))

        m = enc.create_gurobi_model()
        m.setParam('TimeLimit', optim_limit)
        
        if solutionFound:
            m.read(start_file)
            
        m.optimize()

        now = timer()
        while now - t_start < time_limit and (m.SolCount < 0 or (m.SolCount > 0 and m.ObjVal > bestVal - valEps)):
            if not mipBound:
                t0 = timer()
                enc.optimize_constraints(metric=metric, use_lp_relaxation=False)
                t1 = timer()    
                print("MIP-Bounds calculation: {}".format(t1 - t0))
                mipBound = True
                m = enc.create_gurobi_model()
                
                if solutionFound:
                    m.read(start_file)
            
         
            m.setParam('TimeLimit', optim_limit)
            m.optimize()
            now = timer()
            
        # have solution and it is at least valEps better than the last one
        if m.SolCount > 0:
            mipBound = False # if there is better solution now, we can in theory obtain better bounds
            bestVal = m.ObjVal      
            m.write(start_file)
            solutionFound = True
                         
            enc.input_layer.intervars[0].setHi(m.ObjVal + 0.001) # radius
            
            if m.Status == 2:
                # solved to optimality
                break
                
        iteration += 1
 
    return m, enc

In [3]:
clusters_to_verify = pickle.load(open("../to_verify.pickle", "rb"))
cl = clusters_to_verify[1]

center = cl.center

In [4]:
reference = "../ExampleNNs/mnist8x8_lin.h5"
test = "../ExampleNNs/mnist8x8_10p_retrain.h5"
mode = "one_hot_partial_top_2"

In [5]:
input_los = [0 for i in range(64)]
input_his = [16 for i in range(64)]

In [6]:
enc = Encoder()
enc.encode_equiv(reference, test, input_los, input_his, mode)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0
A_pi_3_3 = 0 --> (A_x_2_3 + 0.0001) <= A_top_3_0
A_pi_3_4 = 0 --> (A_x_2_4 + 0.0001) <= A_top_3_0
A_pi_3_5 = 0 --> (A_x_2_5 + 0.0001) <= A_top_3_0
A_pi_3_6 = 0 --> (A_x_2_6 + 0.0001) <= A_top_3_0
A_pi_3_7 = 0 --> (A_x_2_7 + 0.0001) <= A_top_3_0
A_pi_3_8 = 0 --> (A_x_2_8 + 0.0001) <= A_top_3_0
A_pi_3_9 = 0 --> (A_x_2_9 + 0.0001) <= A_top_3_0


In [7]:
enc.add_input_radius(center, 64*16, metric='manhattan', radius_mode='variable')

In [8]:
teststart = timer()
enc.optimize_constraints(metric='manhattan')
now = timer()

Academic license - for non-commercial use only
A_x_0_0: [-79.44250816199929, 90.77746606990695]
A_x_0_1: [-96.4409668520093, 66.7228609919548]
A_x_0_2: [-71.72006513178349, 96.52146522700787]
A_x_0_3: [-66.5850567407906, 84.61267978698015]
A_x_0_4: [-67.48254021070898, 92.98015922680497]
A_x_0_5: [-92.7377373110503, 79.89344880543649]
A_x_0_6: [-68.36480461806059, 78.1730539239943]
A_x_0_7: [-85.99328088760376, 86.14225682243705]
A_x_0_8: [-74.22346387989819, 79.30638812109828]
A_x_0_9: [-69.11462169489823, 82.09353854740039]
A_x_0_10: [-67.62262505106628, 40.98436666466296]
A_x_0_11: [-87.9957521893084, 67.03494120016694]
A_x_0_12: [-82.99049516394734, 96.29432336613536]
A_x_0_13: [-79.38751544756815, 60.877443328965455]
A_x_0_14: [-63.03602630086243, 93.88614222034812]
A_x_0_15: [-96.2067060470581, 51.904001235961914]
A_x_0_16: [-78.90257577225566, 86.56272193416953]
A_x_0_17: [-58.671524830162525, 98.40585792809725]
A_x_0_18: [-66.33490946143866, 53.373301059007645]
A_x_0_19: [-79.9

In [9]:
print("Bounds calculation: {}".format(now - teststart))

Bounds calculation: 35.1310319


In [10]:
model = enc.create_gurobi_model()
model.setParam('TimeLimit', 600)
    
model.optimize()

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 1031 rows, 522 columns and 7337 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [1e-04, 7e+02]
Presolve removed 212 rows and 34 columns
Presolve time: 0.07s
Presolved: 819 rows, 488 columns, 6782 nonzeros
Variable types: 352 continuous, 136 integer (136 binary)

Root relaxation: objective 0.000000e+00, 352 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6          -    0.00000      -     -    0s
     0     0    0.00000    0   11          -    0.00000      -     -    0s
     0     0    0.00000    0   11          -    0.00000      -     -    0s
   

 483176 113301     cutoff   76        29.26227    8.54230  70.8%  23.3  340s
 488936 114442     cutoff   91        29.26227    8.54230  70.8%  23.3  345s
 497251 116276   25.50065   77   11   29.26227    8.57465  70.7%  23.3  350s
 505765 117746     cutoff   75        29.26227    8.61363  70.6%  23.3  355s
 514020 119539   14.86376   85   12   29.26227    8.64281  70.5%  23.3  360s
 520361 121000   26.55701   85    8   29.26227    8.66383  70.4%  23.3  365s
 529164 123747     cutoff   77        29.26227    8.72743  70.2%  23.2  370s
 536509 125847    8.75579   74   27   29.26227    8.72743  70.2%  23.3  375s
 545118 127728   15.07307   87   26   29.26227    8.74543  70.1%  23.3  380s
 552421 129144     cutoff   78        29.26227    8.84222  69.8%  23.2  385s
 559848 130356   24.03204   86    9   29.26227    8.87417  69.7%  23.2  390s
 567384 131631   29.19370   78   23   29.26227    8.88715  69.6%  23.2  395s
 573736 132744   21.48091   88    7   29.26227    8.88887  69.6%  23.2  400s

In [14]:
m1, enc1 = iterative_solving(reference, test, input_los, input_his, mode, center, 'manhattan', 64*16, 
                             valEps=0.016, optim_limit=60)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0
A_pi_3_3 = 0 --> (A_x_2_3 + 0.0001) <= A_top_3_0
A_pi_3_4 = 0 --> (A_x_2_4 + 0.0001) <= A_top_3_0
A_pi_3_5 = 0 --> (A_x_2_5 + 0.0001) <= A_top_3_0
A_pi_3_6 = 0 --> (A_x_2_6 + 0.0001) <= A_top_3_0
A_pi_3_7 = 0 --> (A_x_2_7 + 0.0001) <= A_top_3_0
A_pi_3_8 = 0 --> (A_x_2_8 + 0.0001) <= A_top_3_0
A_pi_3_9 = 0 --> (A_x_2_9 + 0.0001) <= A_top_3_0
A_x_0_0: [-79.44250816199929, 90.77746606990695]
A_x_0_1: [-96.4409668520093, 66.7228609919548]
A_x_0_2: [-71.72006513178349, 96.52146522700787]
A_x_0_3: [-66.5850567407906, 84.61267978698015]
A_x_0_4: [-67.48254021070898, 92.98015922680497]
A_x_0_5: [-92.7377373110503, 79.89344880543649]
A_x_0_6: [-68.36480461806059, 78.1730539239943]
A_x_0_7: [-85.99328088760376, 86.14225682243705]
A_x_0_8: [-74.22346387989819, 79.30638812109828]
A_x_0_9: [-69.11462169489823, 82.09353854

H 2228   983                      50.7411214    0.00000   100%  35.6    5s
H 2230   929                      46.4513485    0.00000   100%  35.5    5s
H 2231   930                      46.4253566    0.00000   100%  35.5    5s
H 2312   889                      38.8469636    0.00000   100%  34.9    5s
H 2981  1302                      38.1329787    0.00000   100%  33.2    6s
  5414  2435   35.03498   48   10   38.13298    0.00000   100%  31.7   10s
H 5415  2435                      38.0833248    0.00000   100%  31.7   10s
* 7452  3300              56      37.1665784    0.00000   100%  30.7   12s
* 7458  3288              55      36.8663384    0.00000   100%  30.7   12s
H 8516  3823                      36.6900875    0.00000   100%  30.4   13s
H 8993  4063                      36.1245080    0.00000   100%  30.4   14s
H 9394  4268                      36.1182795    0.00000   100%  30.3   14s
 10100  4636   12.39827   61   15   36.11828    0.00000   100%  30.0   15s
H10103  4376             

Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Read MIP start from file best_solution.mst
Optimize a model with 911 rows, 522 columns and 7081 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 2e+02]
  RHS range        [1e-04, 1e+02]

Loaded MIP start with objective 29.7037

Presolve removed 241 rows and 80 columns
Presolve time: 0.04s
Presolved: 670 rows, 442 columns, 6461 nonzeros
Variable types: 330 continuous, 112 integer (112 binary)

Root relaxation: objective 0.000000e+00, 324 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6   29.70371    0.00000   100%     -    0s
     0     0    0.00000    0    7   29.70371    0.00000   100%     -  

Bounds calculation: 3.1759142999999312
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Read MIP start from file best_solution.mst
Optimize a model with 911 rows, 522 columns and 7081 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 2e+02]
  RHS range        [1e-04, 1e+02]

Loaded MIP start with objective 29.6868

Presolve removed 241 rows and 80 columns
Presolve time: 0.05s
Presolved: 670 rows, 442 columns, 6461 nonzeros
Variable types: 330 continuous, 112 integer (112 binary)

Root relaxation: objective 0.000000e+00, 310 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6   29.68678    0.00000   100%     -    0s
     0     0    0.00000    0   

B_x_2_0: [-64.30658688104815, 56.881485830343664]
B_x_2_1: [-58.62068029122101, 50.902076210253874]
B_x_2_2: [-78.57091826585173, 67.37416620595151]
B_x_2_3: [-76.36714057788913, 34.86651986159002]
B_x_2_4: [-68.42504318115162, 65.59296467917909]
B_x_2_5: [-87.70950149579997, 38.96010404469243]
B_x_2_6: [-58.48493549680899, 76.42422973228808]
B_x_2_7: [-55.552613353233305, 41.5535728755705]
B_x_2_8: [-41.106621668012544, 50.15085972247467]
B_x_2_9: [-60.82086655826315, 36.209538804632516]
Bounds calculation: 2.9086201000000074
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Read MIP start from file best_solution.mst
Optimize a model with 911 rows, 522 columns and 7081 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 2e+02]
  RHS range        [1e-04, 1e+02]

Loaded MIP start with objective 29.2623

Pr

B_x_1_13: [-24.411035811286595, 25.487259718733274]
B_x_1_14: [-27.82879612788785, 34.83338205657828]
B_x_1_15: [-23.408772958775277, 29.268595898141506]
B_x_2_0: [-62.477527412842164, 55.404839839513144]
B_x_2_1: [-56.93741064936557, 49.63422178408835]
B_x_2_2: [-76.84411765753349, 65.44351593931748]
B_x_2_3: [-74.37611968227412, 33.76204652789967]
B_x_2_4: [-66.51835197540335, 64.15075548993623]
B_x_2_5: [-85.48741071810193, 37.852382611174804]
B_x_2_6: [-56.386049008203514, 75.16816835532853]
B_x_2_7: [-54.02973748535557, 40.505749482476325]
B_x_2_8: [-39.4974492504311, 49.005619121247086]
B_x_2_9: [-59.604066899682486, 34.851389306480385]
Bounds calculation: 2.8236810999999307
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Read MIP start from file best_solution.mst
Optimize a model with 911 rows, 522 columns and 7081 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-

B_x_1_14: [-27.807118818523882, 34.80641662632147]
B_x_1_15: [-23.392311720921754, 29.247341824721357]
B_x_2_0: [-62.428194315247474, 55.36493903158396]
B_x_2_1: [-56.891735618953845, 49.600168535474786]
B_x_2_2: [-76.79740280741257, 65.39112648809949]
B_x_2_3: [-74.32268790956283, 33.73231447483225]
B_x_2_4: [-66.46695718606117, 64.11189129059237]
B_x_2_5: [-85.4274878799414, 37.8225169570957]
B_x_2_6: [-56.329472398291465, 75.13419129180352]
B_x_2_7: [-53.98869984304107, 40.47742867923607]
B_x_2_8: [-39.45393590573603, 48.97470467801928]
B_x_2_9: [-59.5712650794948, 34.814786283968694]
Bounds calculation: 2.843025099999977
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Read MIP start from file best_solution.mst
Optimize a model with 911 rows, 522 columns and 7081 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds ran

B_x_1_11: [8.764756547163202, 33.070765549197986]
B_x_1_12: [-22.559323111025662, 7.303788259070753]
B_x_1_13: [-13.577189959931182, 16.802287570654286]
B_x_1_14: [-19.911306403098425, 26.817205569367882]
B_x_1_15: [-16.715293688499585, 14.4988468797386]
B_x_2_0: [-36.980351603441314, 32.3246172462827]
B_x_2_1: [-35.44140949827799, 33.19296095030266]
B_x_2_2: [-49.34405253949458, 42.55262784384093]
B_x_2_3: [-46.94288097397626, 16.487151535053513]
B_x_2_4: [-38.9062106099134, 42.129873666491186]
B_x_2_5: [-58.87107661936134, 21.60857558728871]
B_x_2_6: [-22.364187991117298, 58.70329310543903]
B_x_2_7: [-31.71233425195779, 28.262884302978105]
B_x_2_8: [-19.804389920607072, 34.492811158499215]
B_x_2_9: [-41.98061208495225, 12.576422060081821]
MIP-Bounds calculation: 10.502133299999969
Read MIP start from file best_solution.mst
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 866 rows, 522 columns and 6986 nonzeros


 433488 143825 infeasible   49        28.46139   13.86586  51.3%  15.5  235s
 442798 147145   21.08901   42   21   28.46139   13.88756  51.2%  15.5  240s

Cutting planes:
  Gomory: 9
  MIR: 16
  Flow cover: 12
  Inf proof: 1

Explored 443304 nodes (6884031 simplex iterations) in 60.01 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 28.4614 28.4614 

Time limit reached
Best objective 2.846138793157e+01, best bound 1.389198493166e+01, gap 51.1901%


In [15]:
m1.setParam('TimeLimit', 1800)
m1.optimize()

Changed value of parameter TimeLimit to 1800.0
   Prev: 60.0  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 866 rows, 522 columns and 6986 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+02]
  RHS range        [1e-04, 9e+01]
Presolved: 634 rows, 424 columns, 6333 nonzeros

Continuing optimization...

 443304 147301     cutoff   53        28.46139   13.89198  51.2%  15.5  240s
 451258 149435     cutoff   49        28.46139   13.95728  51.0%  15.5  245s
 460423 151880   27.97181   62    6   28.46139   14.02151  50.7%  15.5  250s
 470067 154435   16.97186   50   10   28.46139   14.10179  50.5%  15.5  255s
 478899 157025   24.79046   44    8   28.46139   14.15934  50.3%  15.5  260s
 488747 159760   23.51865   45   13   28.46139   14.24447  50.0%  15.5  265s
 497897 162190 infeasible   37        28.46139   14.31554  49.7%  15.5  270s
 505

 1343801 339210   22.28171   71    7   28.46139   18.38350  35.4%  15.5  740s
 1353078 340566     cutoff   60        28.46139   18.41532  35.3%  15.5  745s
 1361603 341811     cutoff   50        28.46139   18.45540  35.2%  15.5  750s
 1370002 343060   25.77564   44    8   28.46139   18.48831  35.0%  15.5  755s
 1378771 344547   18.73208   40    6   28.46139   18.51812  34.9%  15.5  760s
 1386400 346013   27.34596   42    8   28.46139   18.54193  34.9%  15.5  765s
 1394720 347211     cutoff   49        28.46139   18.57099  34.8%  15.5  770s
 1404266 349187     cutoff   45        28.46139   18.59741  34.7%  15.5  775s
 1413811 351018   27.22311   34   14   28.46139   18.62871  34.5%  15.5  780s
 1421240 352127   20.50007   56    7   28.46139   18.65122  34.5%  15.5  785s
 1431122 353670   21.62269   39   13   28.46139   18.68372  34.4%  15.5  790s
 1440897 355328     cutoff   69        28.46139   18.72014  34.2%  15.5  795s
 1450673 357153     cutoff   54        28.46139   18.74831  34.1

 2288862 467916   21.25665   57    6   28.46139   21.02641  26.1%  15.2 1270s
 2297790 468999     cutoff   54        28.46139   21.04295  26.1%  15.2 1275s
 2306967 470016   21.06763   54   16   28.46139   21.06455  26.0%  15.2 1280s
 2315767 471008   23.80161   52   10   28.46139   21.08334  25.9%  15.2 1285s
 2324843 472001     cutoff   43        28.46139   21.10474  25.8%  15.2 1290s
 2333303 472730   26.05710   42    5   28.46139   21.12531  25.8%  15.2 1295s
 2342617 473834   21.14521   55    3   28.46139   21.14498  25.7%  15.2 1300s
 2350627 474697   25.02078   46   20   28.46139   21.16240  25.6%  15.2 1305s
 2359536 476256   23.84293   51    9   28.46139   21.17625  25.6%  15.2 1310s
 2368665 477178   22.26342   44    8   28.46139   21.19655  25.5%  15.2 1315s
 2377020 477955     cutoff   43        28.46139   21.21225  25.5%  15.2 1320s
 2386961 479642     cutoff   52        28.46139   21.23090  25.4%  15.2 1325s
 2395692 481015     cutoff   41        28.46139   21.24745  25.3

 3204836 553560     cutoff   42        28.46139   22.65528  20.4%  15.0 1800s
 3213956 554119     cutoff   52        28.46139   22.66999  20.3%  15.0 1805s
 3222469 554767   22.71988   54   14   28.46139   22.68462  20.3%  14.9 1810s
 3231641 555494   23.12670   49    8   28.46139   22.69878  20.2%  14.9 1815s
 3238535 555729     cutoff   48        28.46139   22.70966  20.2%  14.9 1820s
 3246172 556275   24.38032   37   14   28.46139   22.72147  20.2%  14.9 1825s
 3254984 556791   23.42128   37    4   28.46139   22.73553  20.1%  14.9 1830s
 3264096 557315     cutoff   43        28.46139   22.75081  20.1%  14.9 1835s
 3272976 558191   26.64441   45    9   28.46139   22.76422  20.0%  14.9 1840s
 3281597 558425     cutoff   46        28.46139   22.77733  20.0%  14.9 1845s
 3290779 559032     cutoff   57        28.46139   22.79218  19.9%  14.9 1850s
 3299381 559767 infeasible   56        28.46139   22.80369  19.9%  14.9 1855s
 3307418 560055   22.81791   42   17   28.46139   22.81791  19.8

In [16]:
m1.setParam('TimeLimit', 3600)
m1.optimize()

Changed value of parameter TimeLimit to 3600.0
   Prev: 1800.0  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 866 rows, 522 columns and 6986 nonzeros
Variable types: 386 continuous, 136 integer (136 binary)
Coefficient statistics:
  Matrix range     [1e-04, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+02]
  RHS range        [1e-04, 9e+01]
Presolved: 634 rows, 424 columns, 6333 nonzeros

Continuing optimization...

 3615643 576786   28.45840   45    8   28.46139   23.27061  18.2%  14.8 2063s
 3615979 576795     cutoff   53        28.46139   23.27077  18.2%  14.8 2072s
 3616278 576800   23.28654   47    7   28.46139   23.27128  18.2%  14.8 2078s
 3616616 576814     cutoff   39        28.46139   23.27168  18.2%  14.8 2080s
 3617283 576865   26.06625   42   12   28.46139   23.27271  18.2%  14.8 2085s
 3619027 576922   24.34831   58   17   28.46139   23.27493  18.2%  14.8 2090s
 3622061 577082     cutoff   43        28.46139   23.27960  18.2%  14.8 2

 4351322 601065   24.22081   41   13   28.46139   24.22048  14.9%  14.7 2565s
 4359441 601219   26.87198   69    8   28.46139   24.23073  14.9%  14.7 2570s
 4368194 601134 infeasible   51        28.46139   24.24241  14.8%  14.7 2575s
 4376351 601248   24.93405   50    5   28.46139   24.25241  14.8%  14.7 2580s
 4384528 601242   24.34053   51   18   28.46139   24.26251  14.8%  14.6 2585s
 4391635 601400   25.05114   51   14   28.46139   24.27088  14.7%  14.6 2590s
 4399995 601246     cutoff   46        28.46139   24.28141  14.7%  14.6 2595s
 4409109 601424   25.50954   43   11   28.46139   24.29142  14.7%  14.6 2600s
 4417895 602050   27.34544   58    4   28.46139   24.30011  14.6%  14.6 2605s
 4425751 602243     cutoff   60        28.46139   24.30896  14.6%  14.6 2610s
 4434229 602142     cutoff   43        28.46139   24.31944  14.6%  14.6 2615s
 4442331 602037 infeasible   50        28.46139   24.32892  14.5%  14.6 2620s
 4450896 602068 infeasible   53        28.46139   24.33851  14.5

 5211625 600869   25.14680   43    9   28.46139   25.14680  11.6%  14.4 3095s
 5219573 600690   25.15500   50    7   28.46139   25.15500  11.6%  14.4 3100s
 5227369 600660   28.06555   49   17   28.46139   25.16225  11.6%  14.4 3105s
 5235568 600712     cutoff   57        28.46139   25.16966  11.6%  14.4 3110s
 5243348 600378     cutoff   50        28.46139   25.17754  11.5%  14.4 3115s
 5252205 600907     cutoff   54        28.46139   25.18491  11.5%  14.4 3120s
 5260294 600590     cutoff   41        28.46139   25.19231  11.5%  14.4 3125s
 5268402 600462   25.34067   51   16   28.46139   25.19962  11.5%  14.4 3130s
 5277111 600362 infeasible   37        28.46139   25.20741  11.4%  14.4 3135s
 5286106 600203     cutoff   49        28.46139   25.21628  11.4%  14.4 3140s
 5293129 600116 infeasible   42        28.46139   25.22227  11.4%  14.4 3145s
 5301237 599744   28.45337   43    4   28.46139   25.23029  11.4%  14.4 3150s
 5309513 599317     cutoff   56        28.46139   25.23890  11.3

 6082469 566855     cutoff   61        28.46139   25.94536  8.84%  14.3 3625s
 6089243 566123     cutoff   58        28.46139   25.95197  8.82%  14.3 3630s
 6097289 565483 infeasible   44        28.46139   25.95946  8.79%  14.3 3635s
 6106029 565086     cutoff   58        28.46139   25.96630  8.77%  14.2 3640s
 6114395 564709     cutoff   44        28.46139   25.97339  8.74%  14.2 3645s
 6122804 564262   27.38190   51    5   28.46139   25.98095  8.72%  14.2 3650s
 6131244 563647     cutoff   42        28.46139   25.98819  8.69%  14.2 3655s
 6139577 562834   28.19854   51    7   28.46139   25.99640  8.66%  14.2 3660s
 6146785 562179     cutoff   46        28.46139   26.00300  8.64%  14.2 3665s
 6155287 561576     cutoff   58        28.46139   26.01044  8.61%  14.2 3670s
 6163992 560676     cutoff   51        28.46139   26.01831  8.58%  14.2 3675s
 6172534 560125   27.74777   55   13   28.46139   26.02574  8.56%  14.2 3680s
 6180711 559333     cutoff   60        28.46139   26.03293  8.53

 6962056 479576 infeasible   58        28.46139   26.71384  6.14%  14.1 4155s
 6970759 478597     cutoff   47        28.46139   26.72044  6.12%  14.1 4160s
 6978389 477808   26.72628   52   12   28.46139   26.72628  6.10%  14.1 4165s
 6986980 476508     cutoff   45        28.46139   26.73395  6.07%  14.1 4170s
 6995763 475258     cutoff   52        28.46139   26.74201  6.04%  14.1 4175s
 7003853 473922   26.74946   45    5   28.46139   26.74946  6.01%  14.1 4180s
 7012898 472789     cutoff   56        28.46139   26.75650  5.99%  14.1 4185s
 7021536 471690     cutoff   50        28.46139   26.76348  5.97%  14.1 4190s
 7029768 470338     cutoff   53        28.46139   26.77082  5.94%  14.1 4195s
 7038601 469382     cutoff   43        28.46139   26.77756  5.92%  14.1 4200s
 7046308 468178     cutoff   55        28.46139   26.78478  5.89%  14.1 4205s
 7054703 466764     cutoff   41        28.46139   26.79276  5.86%  14.1 4210s
 7062830 465519     cutoff   36        28.46139   26.80009  5.84

 7848926 311842   28.12896   47   11   28.46139   27.49776  3.39%  13.9 4685s
 7857312 309534     cutoff   47        28.46139   27.50565  3.36%  13.9 4690s
 7865816 307082   27.51424   55   12   28.46139   27.51424  3.33%  13.9 4695s
 7873905 304786     cutoff   53        28.46139   27.52268  3.30%  13.9 4700s
 7882593 302417     cutoff   51        28.46139   27.53089  3.27%  13.9 4705s
 7890789 300189     cutoff   51        28.46139   27.53915  3.24%  13.9 4710s
 7899331 298018     cutoff   55        28.46139   27.54709  3.21%  13.9 4715s
 7908389 295823   27.58761   60    4   28.46139   27.55492  3.18%  13.9 4720s
 7915829 293891     cutoff   56        28.46139   27.56152  3.16%  13.9 4725s
 7924263 291457   27.56954   58   14   28.46139   27.56954  3.13%  13.9 4730s
 7932599 289371     cutoff   48        28.46139   27.57717  3.11%  13.9 4735s
 7940412 287195     cutoff   45        28.46139   27.58492  3.08%  13.9 4740s
 7949373 284960     cutoff   55        28.46139   27.59247  3.05